## Sentinel-2 Satellite Imagery Processing

In this section of the code, we describe how we tapped into Google Earth Engine’s Sentinel-2 archive to quantify land-cover changes for each city and year. First, we define urban, rural, and water masks by applying spectral indices and supervised classification rules. Next, we calculate the total area (in km²) of each mask type—urban footprint, agricultural or undeveloped rural land, and surface water bodies—using pixel counts and the known spatial resolution of Sentinel-2.

By iterating this process for every calendar year in our study period, we create a time series of land-use metrics that capture both expansion (e.g., urban sprawl) and seasonal fluctuations (e.g., flooding or reservoir level changes). At the end of the script, we export each annual dataset as a GeoTIFF or CSV and seamlessly merge it into the master “Base final” database. This integration ensures that our final model not only incorporates economic and demographic variables, but also a dynamic spatial dimension derived from high-resolution remote sensing.  


In [ ]:
import ee
import geemap.core as geemap
from ee import batch
import calendar
import pandas as pd 
import geopandas as gpd
import re
import unicodedata
from shapely.geometry import Point
import time 
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
from geopy.distance import geodesic
from pykrige.kriging_tools import write_asc_grid
from sklearn.preprocessing import MinMaxScaler

# We authenticate on the Google Earth servers
ee.Authenticate()
ee.Initialize(project='ee-iaeconomiaturistas')
print(ee.String('Hello from the Earth Engine servers!').getInfo())


# Functions to use
def arreglar_texto(df , variable , nombre):
    if variable not in df.columns:
        print('La columna' , variable ,'no existe en el DataFrame')
        return df
        
    df[variable] = df[variable].str.lower()
    df[variable] = df[variable].str.replace(' ', '', regex=False)
    df[variable] = df[variable].str.replace(r'[^\w\s]', '', regex=True)   
    df[variable] = df[variable].str.replace('sanandresdetumaco', 'tumaco', regex=False)
    df[variable] = df[variable].str.replace('ct$', '', regex=True)
    df[variable] = df[variable].str.replace('bogotadc', 'bogota', regex=False)
    df[variable] = df[variable].apply(eliminar_tildes)
    df = df.rename(columns={variable: nombre})
    return df

def eliminar_tildes(str):
    return ''.join(
        i for i in unicodedata.normalize('NFKD', str)
        if unicodedata.category(i) != 'Mn')

### We load the database of the cities we are analyzing to join them with their geometries, since this variable only has points

In [2]:

ciudades= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\ciudades.csv')
ciudades['geometry'] = ciudades.apply(lambda i: Point(i['lng'], i['lat']), axis=1)
ciudades= arreglar_texto(ciudades, 'city', 'Ciudad')
ciudades_geometry21= gpd.GeoDataFrame(ciudades, geometry='geometry')
ciudades_geometry21 

,Ciudad,lat,lng,country,iso2,admin_name,capital,population,population_proper,geometry
0,bogota,4.7111,-74.0722,Colombia,CO,Bogotá,primary,7968095,7743955,POINT (-74.0722 4.7111)
1,medellin,6.2308,-75.5906,Colombia,CO,Antioquia,admin,2529403,2529403,POINT (-75.5906 6.2308)
2,cali,3.4206,-76.5222,Colombia,CO,Valle del Cauca,admin,2471474,2471474,POINT (-76.5222 3.4206)
3,barranquilla,10.9833,-74.8019,Colombia,CO,Atlántico,admin,1326588,1274250,POINT (-74.8019 10.9833)
4,cartagena,10.4000,-75.5000,Colombia,CO,Bolívar,admin,914552,914552,POINT (-75.5 10.4)
...,...,...,...,...,...,...,...,...,...,...
79,leticia,-4.2167,-69.9333,Colombia,CO,Amazonas,admin,33503,32450,POINT (-69.9333 -4.2167)
80,lavirginia,4.9167,-75.8333,Colombia,CO,Risaralda,minor,32330,32330,POINT (-75.8333 4.9167)
81,mitu,1.1983,-70.1733,Colombia,CO,Vaupés,admin,28382,28382,POINT (-70.1733 1.1983)
82,inirida,3.8653,-67.9239,Colombia,CO,Guainía,admin,20279,19816,POINT (-67.9239 3.8653)


### We load the polygons of the cities and join them

In [3]:

areas_ciudades = gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Ciudades Colombia Geometry.geojson')
areas_ciudades = arreglar_texto(areas_ciudades , 'Ciudades' , 'Ciudad')
areas_ciudades['geometry_copy'] = areas_ciudades['geometry'].copy()
areas_ciudades = pd.merge(ciudades_geometry21 , areas_ciudades, on='Ciudad', how='left')

### We leave only the variables we need for the analysis

In [4]:

columnas_deseadas = ['Ciudad' , 'geometry_copy']
areas_ciudades = areas_ciudades[columnas_deseadas].dropna()
areas_ciudades

,Ciudad,geometry_copy
0,bogota,"POLYGON ((-74.05613 4.7763, -74.05951 4.77519,..."
1,medellin,"POLYGON ((-75.54463 6.34868, -75.57203 6.34771..."
2,cali,"POLYGON ((-76.52195 3.36074, -76.50713 3.36522..."
3,barranquilla,"POLYGON ((-74.80539 11.03706, -74.82747 11.045..."
4,cartagena,"POLYGON ((-75.46004 10.43033, -75.46269 10.429..."
...,...,...
79,leticia,"POLYGON ((-69.94426 -4.22371, -69.94227 -4.223..."
80,lavirginia,"POLYGON ((-75.87017 4.89399, -75.87276 4.89569..."
81,mitu,"POLYGON ((-70.23955 1.24303, -70.23443 1.2437,..."
82,inirida,"POLYGON ((-67.90653 3.88167, -67.90396 3.88461..."


### We check if there are missing geometries

In [5]:

areas_ciudades[areas_ciudades.isna().any(axis=1)]

,Ciudad,geometry_copy


In [6]:

def otsu(histogram):
    """
    Computes the optimal threshold for image segmentation using Otsu’s method on a histogram.

    Parameters:
        histogram (ee.Dictionary):  
            A dictionary with two entries:
            - 'histogram': list of counts per intensity bin  
            - 'bucketMeans': list of mean intensity values for each bin  

    Returns:
        threshold (float):  
            The intensity value (bin mean) that maximizes the between-class variance,
            representing the optimal cutoff between background and foreground.

    Method Overview:
    1. Convert the input to Earth Engine Array objects for counts and bin centers.
    2. Compute the total pixel count and the global mean intensity.
    3. For each candidate threshold index:
       - Split the histogram into background (bins < threshold) and foreground (bins ≥ threshold).
       - Compute weights (w_b, w_f) as the sum of counts in each class.
       - Compute class means (m_b, m_f) from weighted sums of bin intensities.
       - Calculate the between-class variance:  
         between_var = w_b * w_f * (m_b - m_f)²
    4. Select the bin whose between-class variance is maximized.
    """
    histogram = ee.Dictionary(histogram)
    counts = ee.Array(histogram.get('histogram'))    # Pixel counts per bin
    bins = ee.Array(histogram.get('bucketMeans'))    # Mean intensity per bin

    # Total number of pixels and sum of (count × intensity) for global mean
    total = counts.reduce('sum', [0]).get([0])
    sum_total = counts.multiply(bins).reduce('sum', [0]).get([0])

    # Number of histogram bins
    size = counts.length().get([0])

    def computeBetweenVar(i):
        """
        Compute between-class variance for threshold at bin index i.
        """
        i = ee.Number(i).toInt()

        # Background weight and sum
        w_b = counts.slice(0, 0, i).reduce('sum', [0]).get([0])
        sum_b = counts.slice(0, 0, i) \
                      .multiply(bins.slice(0, 0, i)) \
                      .reduce('sum', [0]) \
                      .get([0])

        # Foreground weight and sum
        w_f = total.subtract(w_b)
        sum_f = sum_total.subtract(sum_b)

        # Class means
        m_b = sum_b.divide(w_b)
        m_f = sum_f.divide(w_f)

        # Between-class variance
        return w_b.multiply(w_f).multiply(m_b.subtract(m_f).pow(2))

    # Evaluate variance for every possible threshold index
    indices = ee.List.sequence(1, size.subtract(1))
    between_vars = indices.map(computeBetweenVar)

    # Select the index with maximum between-class variance
    max_var = ee.Array(between_vars).reduce('max', [0]).get([0])
    threshold_index = between_vars.indexOf(max_var)
    threshold = bins.get([threshold_index])

    return threshold


def calcular_percentil(imagen, banda, percentil, estudio):
    """
    Calculates the specified percentile value for a given band over a study area.

    Parameters:
        imagen (ee.Image):  
            The Earth Engine image to sample.  
        banda (str):  
            The name of the band within the image on which to compute the percentile.  
        percentil (float or int):  
            The desired percentile (e.g., 50 for median, 90 for the 90th percentile).  
        estudio (ee.Geometry):  
            The geometry defining the study area or region of interest.

    Returns:
        ee.Number:  
            The percentile value of the selected band over the specified geometry.

    Method:
    1. Create a percentile reducer for the given percentile.  
    2. Apply `reduceRegion` on the image’s band over the study area:
       - `scale=10` meters per pixel (appropriate for Sentinel-2 resolution).  
       - `maxPixels=1e9` and `bestEffort=True` to handle large regions gracefully.  
    3. Extract the band’s percentile value from the reduction result and return it as an Earth Engine Number.
    """
    reducer = ee.Reducer.percentile([percentil])
    percentil_value = imagen.select(banda).reduceRegion(
        reducer=reducer,
        geometry=estudio,
        scale=10,
        maxPixels=1e9,
        bestEffort=True
    ).get(banda)
    return ee.Number(percentil_value)


# Since our variable is local and represents geometry, we need to convert these city geometries into objects that Google Earth Engine can handle.
# This function checks the geometry type and converts it accordingly.
def geojson_to_ee_geometry(geometry):
    geometry_type = geometry['type']

    if geometry_type == 'Polygon':
        coords = geometry['coordinates']
        return ee.Geometry.Polygon(coords)
    
    elif geometry_type == 'MultiPolygon':
        coords = geometry['coordinates']
        return ee.Geometry.MultiPolygon(coords)
    
    elif geometry_type == 'LineString':
        coords = geometry['coordinates']
        return ee.Geometry.LineString(coords)
    
    elif geometry_type == 'GeometryCollection':
        geometries = geometry['geometries']
        ee_geometries = [geojson_to_ee_geometry(geom) for geom in geometries]
        return ee.FeatureCollection(geometries)
    
    else:
        raise ValueError(f"Unsupported geometry type: {geometry_type}")

s2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

def maskS2clouds(image):
    """
    Applies a cloud mask to a Sentinel-2 image using the S2_CLOUD_PROBABILITY layer.

    Parameters:
        image (ee.Image):  
            A Sentinel-2 image to be masked.

    Returns:
        ee.Image:  
            The input image with cloudy pixels masked out.

    Method:
    1. Retrieve the image’s system index.
    2. Load the corresponding cloud probability image from the COPERNICUS/S2_CLOUD_PROBABILITY collection.
    3. Define an inner function that:
       - Checks if the 'probability' band exists.
       - Creates a mask where probability < 40 (i.e., low cloud likelihood).
       - Applies this mask to the original image and preserves its timestamp metadata.
    4. If a cloud probability image is found, apply the mask; otherwise, return the original image.
    """
    img_id = image.get('system:index')
    cloud_prob = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
        .filter(ee.Filter.eq('system:index', img_id)) \
        .first()

    def apply_cloud_mask(cloud_prob_image):
        cloud_prob_threshold = 40  # Threshold for cloud probability (0–100)
        has_probability = cloud_prob_image.bandNames().contains('probability')
        is_not_cloud = ee.Algorithms.If(
            has_probability,
            cloud_prob_image.select('probability').lt(cloud_prob_threshold),
            ee.Image.constant(1)
        )
        return image.updateMask(is_not_cloud).copyProperties(image, ["system:time_start"])

    return ee.Algorithms.If(cloud_prob, apply_cloud_mask(cloud_prob), image)

# Iniciamios diccionarios para guardar los resultados    
resultados = {}
imagenes_resultados = {}
c=0

## Let's start calculating the urban, rural, and water areas for each of the 81 cities in the years of study

In [14]:
for i in range(len(areas_ciudades)):
    c=c+1
    row = areas_ciudades.iloc[i] 
    geojson_geometry = row['geometry_copy'].__geo_interface__
    area_estudio = geojson_to_ee_geometry(geojson_geometry)
    ciudad = row['Ciudad'] 

    for año in range(2018, 2024):
        if año == 2020: ## We removed 2020 because we are not analyzing it 
            continue
    
        fecha_inicio = f'{año}-01-01'
        fecha_fin = f'{año}-12-31'

        # We created a collection in the study area
        coleccion = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(area_estudio) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) # Since we are using the maskS2clouds mask we can afford to have a high percentage of clouds

        # Filter the collection by date and apply masking, we leave a maximum of 55 images so the code runs faster
        coleccion_año = coleccion.filterDate(fecha_inicio, fecha_fin).map(maskS2clouds).limit(55)

        # Check the number of images available
        num_images = coleccion_año.size().getInfo()
        print(f"Año: {año}, Número de imágenes después del enmascaramiento: {num_images}")

        # If there are no images we complete the dictionary with None and then use KNN 
        if num_images == 0:
            print(f"No hay imágenes disponibles para el año {año} después del enmascaramiento.")
            if ciudad not in resultados:
                resultados[ciudad] = []
            resultados[ciudad].append([año, None , None , None])
            continue

        # We select the bands we need for the different indices 
        bandas_medianas = coleccion_año.select(['B3', 'B4', 'B8', 'B11' , 'B2' , 'B12']).median().clip(area_estudio)
        bandas_medianas = bandas_medianas.multiply(0.0001)
    
        # Calculate NDWI, NDBI, NDVI, GNDVI, MNDWI, EVI and AWEIsh
        
        '''
        Here are the spectral indices we use to characterize land cover and water presence. Each index highlights a different aspect of the surface:

        - **NDVI (Normalized Difference Vegetation Index):**  
        \- Measures vegetation “greenness” by comparing near-infrared (NIR) and red bands.  
        \- Values range from –1 to +1; values closer to +1 indicate denser, healthier plant cover.  

        - **GNDVI (Green Normalized Difference Vegetation Index):**  
        \- A variant of NDVI that uses the green band instead of red, increasing sensitivity to chlorophyll content.  
        \- Particularly useful for monitoring crop health during early growth stages.  

        - **EVI (Enhanced Vegetation Index):**  
        \- Improves upon NDVI by correcting for atmospheric aerosols and soil background.  
        \- Provides better discrimination in areas with very dense canopy cover.  

        - **NDWI (Normalized Difference Water Index):**  
        \- Compares green and NIR bands to detect surface water bodies.  
        \- Helps distinguish open water from land and sparse vegetation.  

        - **MNDWI (Modified NDWI):**  
        \- Replaces NIR with the short-wave infrared (SWIR) band, reducing confusion from built-up areas.  
        \- Enhances water detection in urban and mixed-use landscapes.  

        - **NDBI (Normalized Difference Built-up Index):**  
        \- Uses SWIR and NIR bands to highlight urban development and impervious surfaces.  
        \- Higher values correspond to more extensive human-made structures.  

        - **AWEIsh (Automated Water Extraction Index with Shadow Removal):**  
        \- Automatically identifies water features while minimizing shadow effects, especially in mountainous regions.  
        \- Combines multiple bands to separate true water signals from dark shadows.

        By calculating these indices on Sentinel-2 imagery, we can map vegetation health, urban expansion, and water dynamics with high spatial 
        detail—key inputs for our annual, city-level land-cover dataset.

        '''
        ndwi = bandas_medianas.normalizedDifference(['B3', 'B8']).rename('NDWI')
        ndbi = bandas_medianas.normalizedDifference(['B11', 'B8']).rename('NDBI')
        ndvi = bandas_medianas.normalizedDifference(['B8', 'B4']).rename('NDVI')
        gndvi = bandas_medianas.normalizedDifference(['B8', 'B3']).rename('GNDVI')
        mndwi = bandas_medianas.normalizedDifference(['B3', 'B11']).rename('MNDWI')
        evi = bandas_medianas.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
            {
            'NIR': bandas_medianas.select('B8'),
            'RED': bandas_medianas.select('B4'),
            'BLUE': bandas_medianas.select('B2')
            }
            ).rename('EVI')
        aweish = bandas_medianas.expression(
            'B3 + 2.5 * B8 - 1.5 * (B11 + B12) - 0.25 * B2',
            {
            'B2': bandas_medianas.select('B2'),
            'B3': bandas_medianas.select('B3'),
            'B8': bandas_medianas.select('B8'),
            'B11': bandas_medianas.select('B11'),
            'B12': bandas_medianas.select('B12')
            }
            ).rename('AWEIsh')
        
        # We created an image for each city with the different indices calculated. 
        imagen_compuesta = bandas_medianas.addBands([ndwi, ndbi, ndvi , evi , gndvi , mndwi , aweish])

        # Calculate NDBI histogram to determine optimal threshold
        ndbi_histogram = imagen_compuesta.select('NDBI').reduceRegion(
            reducer=ee.Reducer.histogram(255, 2),  
            geometry=area_estudio,
            scale=10,
            maxPixels=1e9,
            bestEffort=True)
        ndbi_hist = ndbi_histogram.get('NDBI')

    
        # Calculate thresholds using Otsu for NDBI
        if ndbi_histogram:
                ndbi_threshold = otsu(ndbi_hist)
                ndbi_threshold_num = ee.Number(ndbi_threshold)
        else:
            ndbi_threshold = 0.2  # Default value in case otsu cannot be used 
            ndbi_threshold_num = ee.Number(ndbi_threshold)  

        ndvi_threshold_num = calcular_percentil(imagen_compuesta, 'NDVI', 55, area_estudio) # The number represents the percentile we want to calculate
        evi_threshold_num = calcular_percentil(imagen_compuesta, 'EVI', 45, area_estudio) # Since we want to remove the green areas, we calculate a somewhat high NDVI and EVI 

        print(f"Umbrales calculados - NDVI: {ndvi_threshold_num.getInfo()}, EVI: {evi_threshold_num.getInfo()}") # To display the calculated indices


        # We classify urban areas with the NDBI and the optimal threshold of the otsu, removing the green areas calculated by the EVI and NDVI and the water areas from the NDWI.
        areas_urbanas = imagen_compuesta.select('NDBI').gt(ndbi_threshold_num) \
        .And(imagen_compuesta.select('EVI').lt(evi_threshold_num)) \
        .And(imagen_compuesta.select('NDVI').lt(ndvi_threshold_num)) \
        .And(imagen_compuesta.select('NDWI').lt(ee.Number(0.0))) \
        .rename('urban')

        # We recalculate the thresholds for urban areas
        ndvi_threshold_num_R = calcular_percentil(imagen_compuesta, 'NDVI', 86, area_estudio)
        evi_threshold_num_R = calcular_percentil(imagen_compuesta, 'EVI', 81, area_estudio)
        gndvi_threshold_num = calcular_percentil(imagen_compuesta, 'GNDVI', 86, area_estudio)

        # We find the urban areas where all the indicators are high
        zonas_verdes = imagen_compuesta.select('NDVI').gt(ndvi_threshold_num_R) \
            .And(imagen_compuesta.select('EVI').gt(evi_threshold_num_R)) \
            .And(imagen_compuesta.select('GNDVI').gt(gndvi_threshold_num)) \
            .rename('zonas_verdes')

        ## We calculate the water areas
        mndwi_threshold_num = calcular_percentil(imagen_compuesta, 'MNDWI', 91, area_estudio)
        aweish_threshold_num = calcular_percentil(imagen_compuesta, 'AWEIsh', 92, area_estudio)
        ndwi_threshold_num = calcular_percentil(imagen_compuesta, 'NDWI', 91, area_estudio)

        # The same for water area
        areas_agua = imagen_compuesta.select('NDWI').gt(ndwi_threshold_num ) \
            .And(imagen_compuesta.select('MNDWI').gt(mndwi_threshold_num)) \
            .And(imagen_compuesta.select('AWEIsh').gt(aweish_threshold_num)) \
            .rename('agua') 
    
        # Now with the images created we are going to calculate the different urban green and water areas 
        area_pixeles_verdes = zonas_verdes.reduceRegion(
            reducer=ee.Reducer.sum(),geometry=area_estudio,
            scale=10,maxPixels=1e9,bestEffort=True).get('zonas_verdes')

        # Calculate the area of ​​urban areas
        area_pixeles_urbanas = areas_urbanas.reduceRegion(
            reducer=ee.Reducer.sum(),geometry=area_estudio,
            scale=10,maxPixels=1e9,bestEffort=True).get('urban')

        # Calculate the area of ​​urban areas
        area_pixeles_agua = areas_agua.reduceRegion(
            reducer=ee.Reducer.sum(),geometry=area_estudio,
            scale=10,maxPixels=1e9,bestEffort=True).get('agua')


        # Convert to numbers and calculate areas in square kilometers knowing that each pixel is 100 m²
        area_pixeles_verdes = ee.Number(area_pixeles_verdes)
        area_km2_verdes = area_pixeles_verdes.multiply(100).divide(1e6)  # 100 m² per píxel

        area_pixeles_urbanas = ee.Number(area_pixeles_urbanas)
        area_km2_urbanas = area_pixeles_urbanas.multiply(100).divide(1e6)

        area_pixeles_agua = ee.Number(area_pixeles_agua)
        area_km2_agua = area_pixeles_agua.multiply(100).divide(1e6)

        # Get the values ​​and in case of error put None
        try:
            area_km2_valor_verdes = area_km2_verdes.getInfo()
        except:
            print('Error al tratar de obtener area rural para' , ciudad , 'en el ano' ,año)
            area_km2_valor_verdes = None

        try:
            area_km2_valor_urbanas = area_km2_urbanas.getInfo()
        except:
            print('Error al tratar de obtener area urbana para' , ciudad , 'en el ano' ,año)
            area_km2_valor_urbanas = None
        try:
            area_km2_valor_agua = area_km2_agua.getInfo()
        except:
            print('Error al tratar de obtener area agua para' , ciudad , 'en el ano' ,año)
            area_km2_valor_agua = None


        # We add the images and areas to the dictionaries to make them a df later
        if ciudad not in resultados:
            resultados[ciudad] = []

        if ciudad not in imagenes_resultados:
            imagenes_resultados[ciudad] = []

        imagenes_resultados[ciudad].append([año, areas_urbanas , zonas_verdes , areas_agua])
        
        resultados[ciudad].append([año, area_km2_valor_urbanas , area_km2_valor_verdes ,area_km2_valor_agua])

        print(resultados)
        print(c)

# If the code runs completely, it will look something like what was printed below, but with all cities and for all years.

#Year: 2018, Number of images after masking: 15
#Calculated thresholds - NDVI: 0.1835615742969714, EVI: 0.08532861337284849
#{'bogota': [[2018, 189.97533254901958, 37.85034470588236, 0.8084011764705882]]}
#2
#Year: 2019, Number of images after masking: 50 ...

Año: 2018, Número de imágenes después del enmascaramiento: 15
Umbrales calculados - NDVI: 0.1835615742969714, EVI: 0.08532861337284849
{'bogota': [[2018, 189.97533254901958, 37.85034470588236, 0.8084011764705882]]}
2
Año: 2019, Número de imágenes después del enmascaramiento: 50


KeyboardInterrupt: 

### We abort the code to only show the first part of the calculations

### This is the dictionary where the images are stored

In [9]:

imagenes_resultados

{'bogota': [[2018,
   <ee.image.Image at 0x21e233ebb50>]]}

### Verify that what was saved in the dictionary are images

In [16]:

imagenes_resultados['bogota'][0][1] 

###  With this function you can graph the different areas since it is very likely that the code will not run, I put the images that it would throw

In [ ]:

Map = geemap.Map()

visualizacion_areas_urbanas = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'red']  
}
Map.addLayer(imagenes_resultados['bogota'][0][1], visualizacion_areas_urbanas, 'Áreas Urbanas')

Map.centerObject(area_estudio, 10)
Map

### This function allows you to graph different areas. Here we are graphing rural areas.

In [ ]:

Map = geemap.Map()

visualizacion_areas_urbanas = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'red']  
}
Map.addLayer(imagenes_resultados['bogota'][0][2], visualizacion_areas_urbanas, 'Áreas Urbanas')

Map.centerObject(area_estudio, 10)
Map

## Note: Due to GitHub issues, the images were not uploaded directly to the script. However, they can be viewed in the attached Images folder.

In [ ]:
# Volvemos los datos un df 
data_list = []
for ciudad, registros in resultados.items():
    for registro in registros:
        if len(registro) == 4:
            año, area1, area2, area3 = registro
        elif len(registro) == 2:
            año, area1 = registro
            area2, area3 = None, None  
            print(f"Registro inesperado para la ciudad {ciudad}: {registro}")
            año, area1, area2, area3 = None, None, None, None
        data_list.append([ciudad, año, area1, area2, area3])

df = pd.DataFrame(data_list, columns=['ciudad', 'año', 'area1', 'area2', 'area3'])

### We export the database that was exported when the code was run so that it can be seen


In [ ]:

areas = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Areas_Finales.xlsx')

### Let's see how the base we just created was generated:

In [3]:
areas.head(10)

,ciudad,año,Area Urbana,Area Rural,Area Agua
0,bogota,2018,189.975333,37.850345,0.808401
1,bogota,2019,213.094335,53.191889,1.238585
2,bogota,2021,218.924291,54.775529,0.566466
3,bogota,2022,217.850592,52.881088,0.706866
4,bogota,2023,217.187436,54.555076,1.116556
5,medellin,2018,78.806596,19.281878,0.336355
6,medellin,2019,81.961358,20.062621,0.286349
7,medellin,2021,80.073025,18.700133,0.350471
8,medellin,2022,81.386937,19.725720,0.434761
9,medellin,2023,80.515962,19.893864,0.537594


## Pablo Reyes